### Data is the pancreatic cancer biometric data set that I have previously used


In [1]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn import set_config, linear_model
set_config(display="diagram")


import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

In [2]:
raw_data = pd.read_csv('data/Debernardi et al 2020 data.csv')
raw_data.head()


,sample_id,patient_cohort,sample_origin,age,sex,diagnosis,stage,benign_sample_diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,S1,Cohort1,BPTB,33,F,1,NaN,NaN,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,S10,Cohort1,BPTB,81,F,1,NaN,NaN,NaN,0.97266,2.037585,94.46703,209.488250,228.407
2,S100,Cohort2,BPTB,51,M,1,NaN,NaN,7.0,0.78039,0.145589,102.36600,461.141000,NaN
3,S101,Cohort2,BPTB,61,M,1,NaN,NaN,8.0,0.70122,0.002805,60.57900,142.950000,NaN
4,S102,Cohort2,BPTB,62,M,1,NaN,NaN,9.0,0.21489,0.000860,65.54000,41.088000,NaN


In [3]:
clean1 = raw_data[['age','diagnosis','creatinine','LYVE1','REG1B','TFF1']]
clean1 = clean1.dropna()
clean1.head()

,age,diagnosis,creatinine,LYVE1,REG1B,TFF1
0,33,1,1.83222,0.893219,52.94884,654.282174
1,81,1,0.97266,2.037585,94.46703,209.488250
2,51,1,0.78039,0.145589,102.36600,461.141000
3,61,1,0.70122,0.002805,60.57900,142.950000
4,62,1,0.21489,0.000860,65.54000,41.088000


In [4]:
clean1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         590 non-null    int64  
 1   diagnosis   590 non-null    int64  
 2   creatinine  590 non-null    float64
 3   LYVE1       590 non-null    float64
 4   REG1B       590 non-null    float64
 5   TFF1        590 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 27.8 KB


In [5]:
#generate train/test data for auto
X = clean1.drop(['diagnosis'], axis = 1)
y = clean1['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

#### Lasso Method

In [6]:
pipe = Pipeline([('polyfeatures', PolynomialFeatures(degree = 3, include_bias = False)),
                      #('scaler', StandardScaler()),
                     ('lasso', Lasso(random_state = 42))])
pipe.fit(X_train, y_train)
lasso_coefs = pipe.named_steps['lasso'].coef_
pipe

c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.389e+01, tolerance: 2.664e-02
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('polyfeatures',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('lasso', Lasso(random_state=42))])

In [7]:
lasso_train_mse = mean_squared_error(y_train, pipe.predict(X_train))
lasso_test_mse = mean_squared_error(y_test, pipe.predict(X_test))
print(lasso_train_mse)
print(lasso_test_mse)

0.3474266606780669
0.7841167973430052


#### Sequential Features 

In [8]:
feature_names = pipe.named_steps['polyfeatures'].get_feature_names_out()
lasso_df = pd.DataFrame({'feature': feature_names, 'coef': lasso_coefs})
print(lasso_df)

                   feature          coef
0                      age -0.000000e+00
1               creatinine -0.000000e+00
2                    LYVE1  0.000000e+00
3                    REG1B  0.000000e+00
4                     TFF1  1.420082e-03
5                    age^2  5.104088e-04
6           age creatinine -0.000000e+00
7                age LYVE1  0.000000e+00
8                age REG1B  6.604785e-05
9                 age TFF1 -1.661698e-05
10            creatinine^2  0.000000e+00
11        creatinine LYVE1 -0.000000e+00
12        creatinine REG1B -4.252520e-04
13         creatinine TFF1 -3.043311e-04
14                 LYVE1^2  0.000000e+00
15             LYVE1 REG1B  1.654900e-05
16              LYVE1 TFF1 -8.359067e-05
17                 REG1B^2 -2.316861e-07
18              REG1B TFF1  1.620017e-07
19                  TFF1^2 -4.715155e-08
20                   age^3 -5.512254e-06
21        age^2 creatinine -8.095098e-05
22             age^2 LYVE1  7.050995e-05
23             a

In [8]:
sequential_pipe = Pipeline([('poly_features', PolynomialFeatures(degree = 3, include_bias = False)),
                           ('selector', SequentialFeatureSelector(LinearRegression(), 
                                                                  n_features_to_select=6)),
                           ('linreg', LinearRegression())])
sequential_pipe.fit(X_train, y_train)
sequential_train_mse = mean_squared_error(y_train, sequential_pipe.predict(X_train))
sequential_test_mse = mean_squared_error(y_test, sequential_pipe.predict(X_test))

print(sequential_train_mse)
print(sequential_test_mse)

0.4222630595980989
0.39204428836806476


#### Grid Search CV

In [13]:
grid_pipe = Pipeline([('scale', StandardScaler()), ('ridge', Ridge())])
#grid_pipe = Pipeline([('ridge', Ridge())])
param_dict = {'ridge__alpha': [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0]}

In [67]:


grid = GridSearchCV(grid_pipe, param_grid=param_dict, scoring="neg_mean_squared_error", cv=5)
grid.fit(X_train, y_train)
train_preds = grid.predict(X_train)
test_preds = grid.predict(X_test)
model_1_train_mse = mean_squared_error(y_train, train_preds)
model_1_test_mse = mean_squared_error(y_test, test_preds)
model_1_best_alpha = grid.best_params_

print(model_1_train_mse)
print(model_1_test_mse)
print(model_1_best_alpha)
print(grid.cv_results_["mean_test_score"])
print(grid.cv_results_["std_test_score"])

0.43681217811170525
0.41653128684252394
{'ridge__alpha': 10.0}
[-0.45855419 -0.45854635 -0.45847695 -0.45794525 -0.45834777 -0.47344218]
[0.06969176 0.0696783  0.06955789 0.06852318 0.06462561 0.06154991]


In [70]:
model_2_train_mse = ''
model_2_test_mse = ''
model_2_best_alpha = ''
train_preds2 = ''
test_preds2 = ''

grid_2 = GridSearchCV(estimator=grid_pipe, param_grid=param_dict, scoring="neg_mean_squared_error", cv=10)
grid_2.fit(X_train, y_train)
train_preds2 = grid_2.predict(X_train)
test_preds2 = grid_2.predict(X_test)
model_2_train_mse = mean_squared_error(y_train, train_preds2)
model_2_test_mse = mean_squared_error(y_test, test_preds2)
model_2_best_alpha = grid_2.best_params_

print(model_2_train_mse)
print(model_2_test_mse)
print(grid_2.cv_results_["mean_test_score"])
print(grid_2.cv_results_["std_test_score"])

0.43681217811170525
0.41653128684252394
[-0.45740777 -0.45740458 -0.45737666 -0.45719094 -0.45865527 -0.47353946]
[0.08075671 0.08075217 0.08071182 0.08038544 0.07999329 0.08789827]


In [65]:


grid_3 = GridSearchCV(grid_pipe, param_grid=param_dict, scoring="neg_mean_squared_error", cv=413)
grid_3.fit(X_train, y_train)
train_preds3 = grid_3.predict(X_train)
test_preds3 = grid_3.predict(X_test)
model_3_train_mse = mean_squared_error(y_train, train_preds3)
model_3_test_mse = mean_squared_error(y_test, test_preds3)
model_3_best_alpha = grid_3.best_params_

print(model_3_train_mse)
print(model_3_test_mse)
print(grid_3.cv_results_["mean_test_score"])
print(grid_3.cv_results_["std_test_score"])

0.43681217811170525
0.41653128684252394
[-0.45221699 -0.45221488 -0.45219649 -0.45208223 -0.45347764 -0.46484938]
[0.49417909 0.49415329 0.49392202 0.49190416 0.48455174 0.48888925]


In [121]:

lasso = linear_model.Lasso()
num = [5,10,413]

for i in num:
    scores = cross_validate(lasso, X, y, cv=i,
                                scoring=('r2', 'neg_mean_squared_error'),
                                return_train_score=True)

    print(f'Number of folds: {i}')
    print(f'Min Test MSE: {max(scores['test_neg_mean_squared_error'])}')
    print(f'Min Train MSE: {max(scores['train_neg_mean_squared_error'])}')
    print(f'Fit time: {scores['fit_time'].sum()}')
    print(f'Score time: {scores['score_time'].sum()}')
    print(' ')



Number of folds: 5
Min Test MSE: -0.08801722587546723
Min Train MSE: -0.3864496321796888
Fit time: 0.010970592498779297
Score time: 0.007979869842529297
 
Number of folds: 10
Min Test MSE: -0.06039285407420884
Min Train MSE: -0.4620523200166825
Fit time: 0.02593207359313965
Score time: 0.019946813583374023
 


c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Pro

Number of folds: 413
Min Test MSE: -4.2764058420859705e-06
Min Train MSE: -0.4959108328814588
Fit time: 0.7369701862335205
Score time: 0.6073839664459229
 


c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\sspillane\AppData\Local\Pro